In [2]:
import pandas as pd
import numpy as np
import time,datetime
import gc

In [3]:
def datetime2SecondsFrom1970(timeDateStr:str):
    time1=datetime.datetime.strptime(timeDateStr,"%Y-%m-%d %H:%M:%S")
    secondsFrom1970=time.mktime(time1.timetuple())
    return secondsFrom1970

def seconds2Datetime(seconds_from_1970):
    timeArray = time.localtime(seconds_from_1970)#1970秒数
    otherStyleTime = time.strftime("%Y-%m-%d %H:%M:%S", timeArray)
    datetime1=datetime.datetime.strptime(otherStyleTime, "%Y-%m-%d %H:%M:%S")
    return str(datetime1)
def datetime2SecondsFrom1970_s(timeDateStr:str):
    time1=datetime.datetime.strptime(timeDateStr,"%Y%m%d%H%M%S")
    secondsFrom1970=time.mktime(time1.timetuple())
    return int(secondsFrom1970)
def self_split_by_comma(str_):
    return str_.split(',')
def self_split_by_long(str_):
    return str_.split('|')

In [4]:
def date_train_extract_adid_extract(date_train_extract,date_before_day_train_extract):   
    second_need_date_from_1970=datetime2SecondsFrom1970_s('2019'+date_train_extract+'235959')
    ad_static_feature=pd.read_csv("../Data/A_preliminary/testA/ad_static_feature.out",header=None,sep='\t',
                                 names=['ad_id','create_time','ad_account_id','commodity_id','commodity_type','ad_trades_id','ad_size',
                                 ])
    ad_date_train_extract_statice_feature=ad_static_feature[ad_static_feature['create_time']<second_need_date_from_1970]
    ad_date_train_extract_statice_feature.index=range(ad_date_train_extract_statice_feature.shape[0])

    ad_operation=pd.read_csv("../Data/A_preliminary/testA/ad_operation.dat",header=None,sep='\t',
                            names=['ad_id','create_time','operation_type','alter_field','value_alter'])
    ad_operation=ad_operation[ad_operation['create_time']<int('2019'+date_train_extract+'235959')]
    ad_before_date_train_extract_operation_create=ad_operation[ad_operation['operation_type']==2].pivot(index='ad_id',columns='alter_field', values='value_alter')
    ad_before_date_train_extract_operation_create.rename(columns={2:'bid',3:'audience_targeting',4:'time_ad_put'},inplace=True)
    ad_before_date_train_extract_operation_create['ad_id']=ad_before_date_train_extract_operation_create.index
    ad_before_date_train_extract_operation_create.index=range(ad_before_date_train_extract_operation_create.shape[0])

    print(ad_before_date_train_extract_operation_create.info())
    print(ad_date_train_extract_statice_feature.info())
    ad_date_train_create_valid=ad_date_train_extract_statice_feature.merge(ad_before_date_train_extract_operation_create,left_on='ad_id',right_on='ad_id',how='inner')
    ad_date_train_create_valid.fillna(-2,inplace=True)
    ad_date_train_create_valid.drop(ad_date_train_create_valid.index[ad_date_train_create_valid['create_time']==ad_date_train_create_valid['ad_account_id']],inplace=True)
    #这部分ad_id没有在static中创建，或在operation中没有创建操作，在曝光日志中找不到,删除
    ad_date_train_create_valid['exposure_valid']=0
    print(ad_date_train_create_valid.info())

    for index,ad_id in zip(ad_date_train_create_valid.index,ad_date_train_create_valid['ad_id']):
#         print(index)
        if ad_operation[(ad_operation['ad_id']==ad_id)&(ad_operation['alter_field']==1)
                        &(ad_operation['create_time']>int('2019'+date_before_day_train_extract+'235959'))&
                        (ad_operation['value_alter']=='1')].shape[0]>0 :
            #ad_operation['value_alter']是str类型。。。。;
            # date_train_extract 当天有修改为广告有效（alter_field==1,value==1）的操作的 证明该ad_id在 date_train_extract 进入过曝光竞争
            ad_date_train_create_valid['exposure_valid'][index]=1

    ad_operation_less_date_train_extract_00=ad_operation[(ad_operation['create_time']<=int('2019'+date_before_day_train_extract+'235959'))
                                           &(ad_operation['alter_field']==1)&(ad_operation['create_time']>0)]
                                            #在 date_before_day_train_extract 00点以前操作过1字段（启用关闭广告）的记录
    ad_id_most_late_create_time=ad_operation_less_date_train_extract_00.groupby(['ad_id'])['create_time'].aggregate('max')
                                #找出每个id操作的最晚记录
    for index,ad_id in zip(range(ad_id_most_late_create_time.shape[0]),ad_id_most_late_create_time.index):
        #按每个ad_id的最晚操作记录查看其1字段最终的值 为1的是进入曝光竞争的ad_id,否则不是
        temp_index=ad_operation_less_date_train_extract_00['value_alter'].index[(ad_operation_less_date_train_extract_00['ad_id']==ad_id)
                   &(ad_operation_less_date_train_extract_00['create_time']==ad_id_most_late_create_time[ad_id])]
        if 1==int(ad_operation_less_date_train_extract_00['value_alter'][temp_index[0]]):# temp_index[0] 可能有相同id的值？？？！！！
            if ad_date_train_create_valid['exposure_valid'][ad_date_train_create_valid['ad_id']==ad_id].shape[0]>0:
                ad_date_train_create_valid['exposure_valid'][ad_date_train_create_valid['ad_id']==ad_id]=1
#         print(index)

    ad_date_train_create_valid=ad_date_train_create_valid[ad_date_train_create_valid['exposure_valid']>0]
    ad_date_train_create_valid.index=range(ad_date_train_create_valid.shape[0])
    ad_date_train_create_valid.drop(columns='exposure_valid',inplace=True)
    #至此所有参与进date_train_extract广告曝光的cpc的ad_id筛选完毕

    del ad_static_feature
    del ad_operation
    gc.collect()
    print('----------------------------------------------------\n',
          'date_train_extract_adid_extract finished!!\n',
          '----------------------------------------------------')
    return ad_date_train_create_valid

In [5]:
def ad_sample_date_train_extract(ad_date_train_create_valid,date_before_day_train_extract):
    ad_sample_keep_status_time_table=pd.read_csv("../Data/A_preliminary_generate/ad_sample_keep_status_time_table.csv",
                                               sep='\t',header=None,names=['ad_id','bid','audience_targeting','time_ad_put','create_time',
                                                                           'end_time','ad_account_id','commodity_id','commodity_type',
                                                                           'ad_trades_id','ad_size'])
    ad_sample_keep_status_endtime_after_date_train_extract=ad_sample_keep_status_time_table[
        (ad_sample_keep_status_time_table['end_time']>datetime2SecondsFrom1970_s('2019'+date_before_day_train_extract+'235959'))&
    (ad_sample_keep_status_time_table['create_time']<=datetime2SecondsFrom1970_s('2019'+date_train_extract+'235959'))]
#广告属性（样本有效）有效时间必须在 date_train_extract(大于date_before_day_train_extract)内,并且开始时间小于date_train_extract235959 的
    ad_sample_date_train_extract=ad_sample_keep_status_endtime_after_date_train_extract.merge(ad_date_train_create_valid[['ad_id']],on='ad_id',how='inner')
    #date_train_extract内有效的ad_id与样本（ad_id在该时段保持的bid，target,time_put等属性）做交集

    #至此，date_train_extract内能参与曝光的样本准备完毕
    print('----------------------------------------------------\n',
          'ad_sample_date_train_extract finished!!\n',
          '----------------------------------------------------')
    return ad_sample_date_train_extract

In [6]:
def ad_sample_date_train_explosure_extract(ad_sample_date_train_extract,ad_date_train_create_valid,
                                           date_train_extract,date_before_day_train_extract):    
    totalExposureLog=pd.read_csv("../Data/A_preliminary/testA/totalExposureLog.out",header=None,sep='\t',
                                 names=['request_id','ad_request_time','ad_position','user_id','exposure_ad_id','exposure_ad_size','bid','pctr',
                                        'quality_ecpm','total_ecpm'])
    train_date_train_extract_exposurelog=totalExposureLog[
        (totalExposureLog['ad_request_time']>datetime2SecondsFrom1970_s('2019'+date_before_day_train_extract+'235959')) & 
        (totalExposureLog['ad_request_time']<=datetime2SecondsFrom1970_s('2019'+date_train_extract+'235959'))]
    train_date_train_extract_exposurelog.index=range(train_date_train_extract_exposurelog.shape[0])
    #训练集提取日期 date_train_extract 内的曝光日志都找出来。

    print(train_date_train_extract_exposurelog.info())
    print(ad_date_train_create_valid.info())
    train_date_train_extract_exposurelog_valid=train_date_train_extract_exposurelog.merge(
        ad_date_train_create_valid[['ad_id']],left_on='exposure_ad_id',right_on='ad_id',how='inner')
    # date_train_extract 的有效的ad_id的曝光日志取出，其他的是些什么智能出价的非cpc的广告
    print(train_date_train_extract_exposurelog_valid.info())
    
    ad_sample_date_train_extract['exposure_times']=0
    for exposurelog_index,ad_id,ad_request_time in zip(train_date_train_extract_exposurelog_valid.index,
                                                       train_date_train_extract_exposurelog_valid['ad_id'],train_date_train_extract_exposurelog_valid['ad_request_time']):
#         print(exposurelog_index)
        ad_sample_temp_adid=ad_sample_date_train_extract[ad_sample_date_train_extract['ad_id']==ad_id][:]
        for index,start_time,end_time in zip(ad_sample_temp_adid.index,ad_sample_temp_adid['create_time'],ad_sample_temp_adid['end_time']):
            if ad_request_time >=start_time and ad_request_time<end_time:
                ad_sample_date_train_extract['exposure_times'].iloc[index]+=1
                break

    ad_sample_date_train_extract['hold_time']=ad_sample_date_train_extract['end_time']-ad_sample_date_train_extract['create_time']
    ad_sample_date_train_extract.to_csv("../Data/A_preliminary_generate/train_"+date_train_extract+".csv",
                                               sep='\t',index=False,header=None)
    print('--------------------\nAll valid sample\n---------------------\n',ad_sample_date_train_extract.info())

    del totalExposureLog
    gc.collect()
    print('----------------------------------------------------\n',
      'ad_sample_date_train_explosure_extract finished!!\n',
      '----------------------------------------------------')

In [7]:
def audience_orientation_nums_extract(date_train_extract):
    ad_sample=pd.read_csv("../Data/A_preliminary_generate/train_"+date_train_extract+".csv",sep='\t',header=None,names=['ad_id', 'bid', 
            'audience_targeting', 'time_ad_put', 'create_time','end_time', 'ad_account_id', 'commodity_id', 'commodity_type',
           'ad_trades_id', 'ad_size', 'exposure_times', 'hold_time'])
    ad_sample['audience_targeting']=ad_sample['audience_targeting'].apply(self_split_by_long)

    user_data=pd.read_csv("../Data/A_preliminary/testA/user_data",header=None,sep='\t',
                          names=['user_id','age','gender','area','status','education','consuption_ability','device','work','connection_type','behavior'])
    print('load data finished')
    user_data['area']=user_data['area'].apply(self_split_by_comma)
    user_data['status']=user_data['status'].apply(self_split_by_comma)
    user_data['work']=user_data['work'].apply(self_split_by_comma)
    user_data['behavior']=user_data['behavior'].apply(lambda x: self_split_by_comma(x)[:10])
    print(user_data.head(10))

    # #这部分数出窄定向人群数目
    ad_sample['audience_orientation_nums']=0
    full_length=user_data.shape[0]
    audience_targeting_type_nums_dict={}#相同的定向人群不用再去user_data中一一比对，用一个字典存储，有大量重复定向人群
    for sample_index,audience_targeting_list in zip(ad_sample.index,ad_sample['audience_targeting']):
        if audience_targeting_list[0]=='all':#定向为全体人群时
            ad_sample['audience_orientation_nums'].iloc[sample_index]=full_length
            print(sample_index,full_length)
            continue
        audience_targeting_str='|'.join(audience_targeting_list)
        if audience_targeting_str in audience_targeting_type_nums_dict:
            #有定向要求相同的、不同出价的广告样本，以及同类型的定向人群相同的广告样本。不需要再去数一次，存入一个dict中，若相同直接载入
            ad_sample['audience_orientation_nums'].iloc[sample_index]=audience_targeting_type_nums_dict[audience_targeting_str]
            print(sample_index,audience_targeting_type_nums_dict[audience_targeting_str])
            continue
        else:
            target_type_content_dict={}
            for target_type_content in audience_targeting_list:
                target_type_content_list=target_type_content.split(':')
                target_type_content_dict[target_type_content_list[0]]=target_type_content_list[1].split(',')
                #如{'age':[787,753,601,202,229,333,741,819,479,608,433,394], 'gender':2}所有的定向要求变成dict，方便查询
            user_data['check']=True#新的check列，符合要求为True，否则False
            for target_check in target_type_content_dict.keys():
                if target_check =='age':
                    temp_set=set(target_type_content_dict['age'])
                    user_data['check']=(user_data['check'])&(user_data['age'].apply(lambda x:set([str(x)])&temp_set)!=set())
                elif target_check =='gender':
                    temp_set=set(target_type_content_dict['gender'])
                    user_data['check']=(user_data['check'])&(user_data['gender'].apply(lambda x:set([str(x)])&temp_set)!=set())
                elif target_check =='education':
                    temp_set=set(target_type_content_dict['education'])
                    user_data['check']=(user_data['check'])&(user_data['education'].apply(lambda x:set([str(x)])&temp_set)!=set())
                elif target_check =='consuption_ability':
                    temp_set=set(target_type_content_dict['consuption_ability'])
                    user_data['check']=(user_data['check'])&(user_data['consuption_ability'].apply(lambda x:set([str(x)])&temp_set)!=set())
                elif target_check =='device':
                    temp_set=set(target_type_content_dict['device'])
                    user_data['check']=(user_data['check'])&(user_data['device'].apply(lambda x:set([str(x)])&temp_set)!=set())
                elif target_check =='connection_type':
                    temp_set=set(target_type_content_dict['connection_type'])
                    user_data['check']=(user_data['check'])&(user_data['connection_type'].apply(lambda x:set([str(x)])&temp_set)!=set())
                #以上字段,用户属性只能一个的，如最高教育程度等，按定向要求的分组互相取并集获得check列
                elif target_check=='area':
                    temp_set=set(target_type_content_dict['area'])
                    user_data['check']=(user_data['check'])&(user_data['area'].apply(lambda x:set(x)&temp_set)!=set())
        #             print('area_check',sum(user_data['area_check']))
                elif target_check=='status':
                    temp_set=set(target_type_content_dict['status'])
                    user_data['check']=(user_data['check'])&(user_data['status'].apply(lambda x:set(x)&temp_set)!=set())
        #             print('status_check',sum(user_data['status_check']))
                elif target_check=='work':
                    temp_set=set(target_type_content_dict['work'])
                    user_data['check']=(user_data['check'])&(user_data['work'].apply(lambda x:set(x)&temp_set)!=set())
                elif target_check=='behavior':
                    temp_set=set(target_type_content_dict['behavior'])
                    user_data['check']=(user_data['check'])&(user_data['behavior'].apply(lambda x:set(x)&temp_set)!=set())
                #这部分字段属于 用户属性可以有多个，同时定向要求可以有多个，将每项两部分作为set取交，交集不为空即符合check，=1
            audience_orientation_nums=user_data[user_data['check']==True].shape[0] 
            ad_sample['audience_orientation_nums'].iloc[sample_index]=audience_orientation_nums
            print(sample_index,audience_orientation_nums)
        audience_targeting_type_nums_dict[audience_targeting_str]=audience_orientation_nums
    #     gc.collect()
    #     获得窄定向人群数量
    ad_sample['audience_targeting']=ad_sample['audience_targeting'].apply(lambda x: '|'.join(x))#把人群定向列转换为原始的字符串，数据格式统一

    del user_data
    gc.collect()
    print('----------------------------------------------------\n',
      'audience_orientation_nums_extract finished!!\n',
      '----------------------------------------------------')
    return ad_sample

In [10]:
def ad_sample_train_time_put(ad_sample,date_train_extract,week_day):
    print('time_put')
    ad_sample['time_ad_put']=ad_sample['time_ad_put'].apply(self_split_by_comma)
    ad_sample['time_ad_put']=ad_sample['time_ad_put'].apply(lambda x:x[week_day])
    ad_sample['sample_id']=ad_sample.index
    time_ad_put_each_halfhour=ad_sample[['sample_id','time_ad_put']].copy()

    time_48_array=np.zeros((1,48), dtype=int)
    for sample_id_index,time_ad_put in zip(time_ad_put_each_halfhour['sample_id'],time_ad_put_each_halfhour['time_ad_put']):
#         print(sample_id_index)
        char_time_put_flag_list=list(str(bin(int(time_ad_put)))[2:])
        int_time_put_flag_list=list(map(int,char_time_put_flag_list))
        int_time_put_flag_list=[0]*(48-len(char_time_put_flag_list))+int_time_put_flag_list
        time_48_array=np.concatenate((time_48_array,np.array(int_time_put_flag_list).reshape(1,48)),axis=0)
        #将广告投放时间按48份格式 转为相应的48维特征
    time_48_array=time_48_array[1:]
    #获取各时间段列名
    list_a=[]
    for _ in range(48):
        if _%2==0:
            start_minutes='00'
            end_minutes='30'
        else:
            start_minutes='30'
            end_minutes='00'
        start_hour='0'+str(_//2)
        end_hour='0'+str(_//2+_%2)
        col_na=start_hour[-2:]+start_minutes+'_'+end_hour[-2:]+end_minutes
        list_a.append(col_na)

    sample_time_halfhour=pd.DataFrame(time_48_array,columns=list_a)
    sample_time_halfhour['sample_id']=ad_sample.index
    ad_sample_train=pd.merge(ad_sample,sample_time_halfhour,on='sample_id',how='inner')
    ad_sample_train.drop(columns='time_ad_put',inplace=True)
    print('write to disk')
    ad_sample_train.to_csv('../Data/A_preliminary_generate/Train/ad_sample_'+date_train_extract+'_train.csv',index=False)
    print('----------------------------------------------------\n',
      'ad_sample_train_time_put finished!!\n',
      '----------------------------------------------------')
    #至此原始特征提取完毕

In [11]:
def train_data_extract_by_date(date_train_extract,date_before_day_train_extract,week_day):
    df_ad_date_train_create_valid=date_train_extract_adid_extract(date_train_extract,date_before_day_train_extract)
    #所有参与进date_train_extract广告曝光的cpc的ad_id筛选完毕
    df_ad_sample_date_train_extract=ad_sample_date_train_extract(df_ad_date_train_create_valid,date_before_day_train_extract)
    ad_sample_date_train_explosure_extract(df_ad_sample_date_train_extract,df_ad_date_train_create_valid,
                                           date_train_extract, date_before_day_train_extract)
    df_ad_sample=audience_orientation_nums_extract(date_train_extract)
    ad_sample_train_time_put(df_ad_sample,date_train_extract,week_day)

In [14]:
date_train_extract='0309'#训练集特征提取的对应日期
date_before_day_train_extract='0308'#对应日期的前一天
week_day=5#周几，用于提取时间定向特征，周一为0，周二为1...
train_data_extract_by_date(date_train_extract,date_before_day_train_extract,week_day)

/home/zhangyier/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2963: DtypeWarning: Columns (3,6) have mixed types. Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37321 entries, 0 to 37320
Data columns (total 4 columns):
bid                   37321 non-null object
audience_targeting    37321 non-null object
time_ad_put           37321 non-null object
ad_id                 37321 non-null int64
dtypes: int64(1), object(3)
memory usage: 1.1+ MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 455158 entries, 0 to 455157
Data columns (total 7 columns):
ad_id             455158 non-null int64
create_time       455158 non-null int64
ad_account_id     455158 non-null int64
commodity_id      455158 non-null object
commodity_type    455158 non-null int64
ad_trades_id      455158 non-null object
ad_size           373952 non-null object
dtypes: int64(4), object(3)
memory usage: 24.3+ MB
None
<class 'pandas.core.frame.DataFrame'>
Int64Index: 30760 entries, 0 to 30759
Data columns (total 11 columns):
ad_id                 30760 non-null int64
create_time           30760 non-null int64
ad_account_id     

/home/zhangyier/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/zhangyier/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


----------------------------------------------------
 date_train_extract_adid_extract finished!!
 ----------------------------------------------------
----------------------------------------------------
 ad_sample_date_train_extract finished!!
 ----------------------------------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3809474 entries, 0 to 3809473
Data columns (total 10 columns):
request_id          int64
ad_request_time     int64
ad_position         int64
user_id             int64
exposure_ad_id      int64
exposure_ad_size    int64
bid                 int64
pctr                float64
quality_ecpm        float64
total_ecpm          float64
dtypes: float64(3), int64(7)
memory usage: 290.6 MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10781 entries, 0 to 10780
Data columns (total 10 columns):
ad_id                 10781 non-null int64
create_time           10781 non-null int64
ad_account_id         10781 non-null int64
commodity_id          10781

/home/zhangyier/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:189: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


...

11651 30719
11652 7240
11653 7240
11654 30719
11655 30719
11656 30719
11657 5709
11658 1197688
11659 1048292
11660 10941
11661 12654
11662 12654
11663 1048292
11664 1396718
11665 1396718
11666 1048292
11667 1396718
11668 1396718
11669 0
11670 0
11671 0
11672 1048292
11673 1048292
11674 30719
11675 793942
11676 1048292
11677 30719
11678 25774
11679 18377
11680 18377
11681 1048292
11682 1048292
11683 3527
11684 3527
11685 1048292
11686 1048292
11687 1048292
11688 1396718
11689 1396718
11690 30719
11691 30719
11692 1048292
11693 1048292
11694 30719
11695 33211
11696 9337
11697 9337
11698 1048292
11699 1048292
11700 33136
11701 33136
11702 30719
11703 1396718
11704 5015
11705 5015
11706 1048292
11707 399503
11708 1048292
11709 33136
11710 7261
11711 1396718
11712 163609
11713 1048292
11714 1048292
11715 6387
11716 6387
11717 12125
11718 1048292
11719 1048292
11720 1048292
11721 45296
11722 10071
11723 1048292
11724 1048292
11725 1048292
11726 1048292
11727 30719
11728 30719
11729 25774
117

12308 2156
12309 1048292
12310 1048292
12311 33211
12312 33211
12313 10941
12314 10941
12315 30719
12316 30719
12317 33136
12318 33136
12319 2299
12320 1048292
12321 30719
12322 1048292
12323 1048292
12324 1048292
12325 30719
12326 38448
12327 30719
12328 1048292
12329 1048292
12330 30719
12331 30719
12332 30719
12333 30719
12334 1048292
12335 1048292
12336 30719
12337 33136
12338 33136
12339 30719
12340 33211
12341 33211
12342 1396718
12343 1048292
12344 1048292
12345 3797
12346 507
12347 1396718
12348 7261
12349 7261
12350 182
12351 1048292
12352 892
12353 1396718
12354 1396718
12355 1048292
12356 1048292
12357 30719
12358 1048292
12359 30719
12360 1396718
12361 1048292
12362 235757
12363 592
12364 3987
12365 1048292
12366 1048292
12367 30719
12368 1048292
12369 14338
12370 1048292
12371 1048292
12372 1048292
12373 1396718
12374 1396718
12375 1197688
12376 1048292
12377 1048292
12378 12654
12379 30719
12380 30719
12381 3358
12382 3358
12383 30719
12384 30719
12385 30719
12386 5057
12

12962 10838
12963 10838
12964 1396718
12965 1048292
12966 1391154
12967 1396718
12968 1396718
12969 1396718
12970 1048292
12971 1048292
12972 1048292
12973 1078
12974 623
12975 2815
12976 3574
12977 0
12978 3574
12979 3574
12980 1048292
12981 30719
12982 14338
12983 14338
12984 21
12985 1048292
12986 802
12987 30719
12988 1048292
12989 1396718
12990 1197688
12991 30719
12992 783
12993 176
12994 1396718
12995 1396718
12996 235757
12997 15627
12998 30719
12999 30719
13000 1048292
13001 1396718
13002 1396718
13003 10
13004 1396718
13005 1396718
13006 13653
13007 29413
13008 30719
13009 12125
13010 12125
13011 30719
13012 1048292
13013 1048292
13014 30719
13015 2451
13016 30719
13017 30719
13018 1048292
13019 31680
13020 31680
13021 30589
13022 1048292
13023 44783
13024 1048292
13025 1048292
13026 18377
13027 1349116
13028 30589
13029 30589
13030 15627
13031 1396718
13032 1048292
13033 15627
13034 15627
13035 30719
13036 1396718
13037 1048292
13038 30719
13039 1048292
13040 1048292
13041 1

13609 14338
13610 14338
13611 1197688
13612 1048292
13613 1048292
13614 33211
13615 33211
13616 1048292
13617 62
13618 30719
13619 1349116
13620 235757
13621 1048292
13622 1048292
13623 1048292
13624 1048292
13625 1048292
13626 11838
13627 98
13628 916
13629 1035427
13630 30719
13631 30719
13632 0
13633 30719
13634 1396718
13635 180162
13636 20008
13637 20008
13638 1197688
13639 1197688
13640 312209
13641 1048292
13642 30719
13643 30719
13644 258
13645 25774
13646 25774
13647 10159
13648 162844
13649 33136
13650 33136
13651 33136
13652 12654
13653 12654
13654 30719
13655 1396718
13656 1396718
13657 18415
13658 14338
13659 1396718
13660 33211
13661 14338
13662 14338
13663 1048292
13664 1396718
13665 1396718
13666 1048292
13667 401209
13668 1396718
13669 1396718
13670 1048292
13671 1048292
13672 592
13673 2480
13674 498987
13675 33211
13676 12125
13677 31680
13678 12654
13679 12654
13680 1048292
13681 1048292
13682 1396718
13683 30719
13684 30719
13685 1048292
13686 3987
13687 3759
13688

14258 1048292
14259 1048292
14260 1048292
14261 1048292
14262 25774
14263 25774
14264 31680
14265 31680
14266 23279
14267 1048292
14268 1048292
14269 33136
14270 33136
14271 9518
14272 9518
14273 25774
14274 25774
14275 1048292
14276 1396718
14277 1392631
14278 1048292
14279 1048292
14280 1048292
14281 1048292
14282 1048292
14283 1048292
14284 16272
14285 30719
14286 30719
14287 1048292
14288 1048292
14289 1048292
14290 1048292
14291 1048292
14292 1396718
14293 1396718
14294 25774
14295 25774
14296 25774
14297 25774
14298 30719
14299 30719
14300 1396718
14301 1048292
14302 1048292
14303 1048292
14304 1048292
14305 12654
14306 956
14307 1396718
14308 30719
14309 3527
14310 1048292
14311 5057
14312 2082
14313 1048292
14314 1396718
14315 1396718
14316 1396718
14317 1048292
14318 1048292
14319 30719
14320 1048292
14321 0
14322 1681
14323 1681
14324 1681
14325 1681
14326 1681
14327 33136
14328 33136
14329 1396718
14330 1048292
14331 33211
14332 33211
14333 1048292
14334 1048292
14335 104829

In [13]:
date_train_extract_list=['0309','0308']
date_before_day_train_extract_list=['0308','0307']
week_day_list=[5,4]
for date_train_extract,date_before_day_train_extract,week_day in zip(date_train_extract_list,date_before_day_train_extract_list,week_day_list):
    print('--------------------------------------------------\n',date_train_extract,'\n--------------------------------------------------')
    train_data_extract_by_date(date_train_extract,date_before_day_train_extract,week_day)

--------------------------------------------------
 0311 
--------------------------------------------------


/home/zhangyier/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2963: DtypeWarning: Columns (3,6) have mixed types. Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37321 entries, 0 to 37320
Data columns (total 4 columns):
bid                   37321 non-null object
audience_targeting    37321 non-null object
time_ad_put           37321 non-null object
ad_id                 37321 non-null int64
dtypes: int64(1), object(3)
memory usage: 1.1+ MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 480884 entries, 0 to 480883
Data columns (total 7 columns):
ad_id             480884 non-null int64
create_time       480884 non-null int64
ad_account_id     480884 non-null int64
commodity_id      480884 non-null object
commodity_type    480884 non-null int64
ad_trades_id      480884 non-null object
ad_size           395628 non-null object
dtypes: int64(4), object(3)
memory usage: 25.7+ MB
None
<class 'pandas.core.frame.DataFrame'>
Int64Index: 31788 entries, 0 to 31787
Data columns (total 11 columns):
ad_id                 31788 non-null int64
create_time           31788 non-null int64
ad_account_id     

/home/zhangyier/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/zhangyier/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


----------------------------------------------------
 date_train_extract_adid_extract finished!!
 ----------------------------------------------------
----------------------------------------------------
 ad_sample_date_train_extract finished!!
 ----------------------------------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3046077 entries, 0 to 3046076
Data columns (total 10 columns):
request_id          int64
ad_request_time     int64
ad_position         int64
user_id             int64
exposure_ad_id      int64
exposure_ad_size    int64
bid                 int64
pctr                float64
quality_ecpm        float64
total_ecpm          float64
dtypes: float64(3), int64(7)
memory usage: 232.4 MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10730 entries, 0 to 10729
Data columns (total 10 columns):
ad_id                 10730 non-null int64
create_time           10730 non-null int64
ad_account_id         10730 non-null int64
commodity_id          10730

...

12259 30719
12260 25774
12261 16447
12262 33211
12263 33211
12264 1394218
12265 1048292
12266 1048292
12267 31680
12268 10941
12269 1048292
12270 1048292
12271 1396718
12272 1048292
12273 1048292
12274 1048292
12275 1396718
12276 1396718
12277 30719
12278 7240
12279 30719
12280 30719
12281 30719
12282 5709
12283 1197688
12284 1197688
12285 1048292
12286 1048292
12287 10941
12288 10941
12289 12654
12290 12654
12291 1048292
12292 1396718
12293 1048292
12294 1396718
12295 1396718
12296 0
12297 1048292
12298 1048292
12299 1048292
12300 30719
12301 793942
12302 1048292
12303 30719
12304 25774
12305 18377
12306 1048292
12307 1048292
12308 3527
12309 1048292
12310 1048292
12311 1048292
12312 1048292
12313 1396718
12314 1396718
12315 30719
12316 30719
12317 1048292
12318 1048292
12319 30719
12320 592
12321 33211
12322 9337
12323 1048292
12324 1048292
12325 33136
12326 33136
12327 30719
12328 1396718
12329 5015
12330 399503
12331 1048292
12332 33136
12333 7261
12334 7261
12335 1396718
12336 163

12908 30719
12909 30719
12910 1048292
12911 1048292
12912 1048292
12913 1783
12914 1048292
12915 1048292
12916 31680
12917 1048292
12918 12654
12919 1197688
12920 1197688
12921 9518
12922 1048292
12923 1048292
12924 30719
12925 9337
12926 6299
12927 745708
12928 1048292
12929 1048292
12930 1197688
12931 1197688
12932 1048292
12933 156610
12934 33211
12935 33211
12936 1396718
12937 1396718
12938 7366
12939 2156
12940 1048292
12941 1048292
12942 33211
12943 33211
12944 10941
12945 30719
12946 30719
12947 1396718
12948 33136
12949 33136
12950 2299
12951 1048292
12952 30719
12953 30719
12954 1048292
12955 1048292
12956 1048292
12957 30719
12958 38448
12959 30719
12960 30719
12961 1048292
12962 1048292
12963 30719
12964 30719
12965 30719
12966 30719
12967 30719
12968 1048292
12969 1396718
12970 30719
12971 30719
12972 33136
12973 33136
12974 14338
12975 33211
12976 31680
12977 31680
12978 33211
12979 1396718
12980 1048292
12981 1048292
12982 3797
12983 3291
12984 3291
12985 507
12986 139671

13561 1197688
13562 30719
13563 30719
13564 348
13565 1048292
13566 30719
13567 30589
13568 1048292
13569 33136
13570 33136
13571 1396718
13572 1396718
13573 1048292
13574 1048292
13575 1349116
13576 1396718
13577 1396718
13578 1392
13579 25774
13580 25774
13581 30719
13582 30719
13583 1396718
13584 1396718
13585 30719
13586 1396718
13587 1396718
13588 7775
13589 1048292
13590 1048292
13591 15884
13592 15884
13593 1048292
13594 1048292
13595 33136
13596 33136
13597 30719
13598 30719
13599 30719
13600 1048292
13601 1048292
13602 75750
13603 30589
13604 30589
13605 1048292
13606 1048292
13607 1396718
13608 30719
13609 30719
13610 1048292
13611 1048292
13612 1048292
13613 1048292
13614 1048292
13615 12654
13616 25774
13617 86744
13618 33211
13619 33211
13620 1382188
13621 235757
13622 1048292
13623 10159
13624 10838
13625 10838
13626 1396718
13627 1048292
13628 1391154
13629 1396718
13630 1396718
13631 1396718
13632 1048292
13633 1048292
13634 1048292
13635 1048292
13636 3574
13637 30719


14199 1394218
14200 31680
14201 31680
14202 30719
14203 30719
14204 15627
14205 1048292
14206 1048292
14207 1048292
14208 33211
14209 33211
14210 30719
14211 30719
14212 12654
14213 12654
14214 26
14215 1048292
14216 30719
14217 1197688
14218 1396718
14219 1396718
14220 17264
14221 1197688
14222 1197688
14223 33211
14224 33211
14225 30719
14226 45296
14227 1048292
14228 1048292
14229 12654
14230 5709
14231 5709
14232 9873
14233 1048292
14234 1048292
14235 1048292
14236 1048292
14237 0
14238 33211
14239 6203
14240 1396718
14241 1396718
14242 23279
14243 25774
14244 745708
14245 1048292
14246 1048292
14247 18415
14248 1048292
14249 1048292
14250 1
14251 1048292
14252 16272
14253 16272
14254 30719
14255 3797
14256 3797
14257 12654
14258 12654
14259 30719
14260 30719
14261 30719
14262 30719
14263 29699
14264 1048292
14265 1048292
14266 1151
14267 1396718
14268 34971
14269 299
14270 1392631
14271 30719
14272 30719
14273 14338
14274 14338
14275 1048292
14276 33211
14277 33211
14278 1048292
1

14857 11948
14858 1048292
14859 1396718
14860 1396718
14861 30589
14862 30589
14863 1349116
14864 30719
14865 30719
14866 33136
14867 33136
14868 1396718
14869 1048292
14870 1048292
14871 86744
14872 1048292
14873 1048292
14874 1048292
14875 33211
14876 33211
14877 16272
14878 16272
14879 15884
14880 1048292
14881 1048292
14882 1048292
14883 1048292
14884 11948
14885 11948
14886 25774
14887 25774
14888 1048292
14889 938
14890 33211
14891 33211
14892 1048292
14893 33136
14894 33136
14895 30719
14896 18377
14897 18377
14898 1197688
14899 1197688
14900 1048292
14901 1108301
14902 1048292
14903 1396718
14904 1396718
14905 1396718
14906 30719
14907 30719
14908 30719
14909 262
14910 262
14911 1048292
14912 1396718
14913 1396718
14914 33211
14915 33211
14916 33211
14917 1048292
14918 1048292
14919 1048292
14920 1048292
14921 8645
14922 8645
14923 1048292
14924 1048292
14925 1048292
14926 25774
14927 25774
14928 31680
14929 31680
14930 23279
14931 1048292
14932 1048292
14933 33136
14934 33136
